In [64]:
import os
import gc
import ot
import pickle
import anndata
import scanpy as sc
import pandas as pd
import numpy as np
from scipy import sparse
from scipy.stats import spearmanr, pearsonr
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt

import commot as ct

## Preprocessing

In [63]:
path_L1_1 = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/xenium_objs/anndata/lung_L1_1.h5ad"

L1_1 = sc.read_h5ad(path_L1_1)

path_chuvio_L1_1 = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/xenium_objs/anndata/chuvio_L1_1.h5ad"

chuvio_L1_1 = sc.read_h5ad(path_chuvio_L1_1)

In [38]:
adata = L1_1
adata_chuv = chuvio_L1_1

In [39]:
adata.obsm['spatial'] = adata.obs[['x_centroid', 'y_centroid']].to_numpy()
adata_chuv.obsm['spatial'] = adata_chuv.obs[['x_centroid', 'y_centroid']].to_numpy()

In [40]:
adata.raw = adata
adata_chuv.raw = adata_chuv
#preprocessing by normalisation through log1p has already been done. 

In [41]:
adata

AnnData object with n_obs × n_vars = 72666 × 271
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'libsize_drop', 'sizeFactor', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'singler_annotation', 'ident'
    uns: 'X_name'
    obsm: 'PCA', 'TSNE', 'UMAP', 'spatial'
    layers: 'logcounts'

In [42]:
adata_dis500 = adata.copy()
adata_chuv_dis500 = adata_chuv.copy()

In [43]:
#What is this doing ?
#sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
#adata = adata[:, adata.var.highly_variable]
#sc.tl.pca(adata, svd_solver='arpack')
#sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
#sc.tl.umap(adata)

In [44]:
adata

AnnData object with n_obs × n_vars = 72666 × 271
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'libsize_drop', 'sizeFactor', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'singler_annotation', 'ident'
    uns: 'X_name'
    obsm: 'PCA', 'TSNE', 'UMAP', 'spatial'
    layers: 'logcounts'

In [45]:
## Plotting the tissue through scanpy
#sc.pl.spatial(adata, color='singler_annotation')

## Spatial communication inference

Using CellChat ligand-receptor database (only other option is CellPhoneDB_v4.0), try using the Liana one that is more complete ? 

In [46]:
df_cellchat = ct.pp.ligand_receptor_database(species='human', signaling_type='Secreted Signaling', database='CellChat')
print(df_cellchat.shape)

(1199, 4)


In [47]:
df_cellchat_filtered = ct.pp.filter_lr_database(df_cellchat, adata_dis500, min_cell_pct=0.00001)
print(df_cellchat_filtered.shape)

(8, 4)


In [53]:
#barely more interesting ligand receptors found with the chuvio panel
df_cellchat_filtered_chuv = ct.pp.filter_lr_database(df_cellchat, adata_chuv_dis500, min_cell_pct=0.00000001)
print(df_cellchat_filtered_chuv.shape)

(34, 4)


In [54]:
df_cellchat_filtered

,0,1,2,3
0,AREG,EGFR,EGF,Secreted Signaling
1,CXCL5,ACKR1,CXCL,Secreted Signaling
2,CXCL6,ACKR1,CXCL,Secreted Signaling
3,CXCL9,ACKR1,CXCL,Secreted Signaling
4,CXCL10,ACKR1,CXCL,Secreted Signaling
5,CXCL13,ACKR1,CXCL,Secreted Signaling
6,CXCL13,CXCR5,CXCL,Secreted Signaling
7,FASLG,FAS,FASLG,Secreted Signaling


In [27]:
adata_dis500

AnnData object with n_obs × n_vars = 72666 × 271
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'libsize_drop', 'sizeFactor', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'singler_annotation', 'ident'
    uns: 'X_name'
    obsm: 'PCA', 'TSNE', 'UMAP', 'spatial'
    layers: 'logcounts'

In [1]:
ct.tl.spatial_communication(adata_dis500,
    database_name='cellchat', df_ligrec=df_cellchat_filtered, dis_thr=500, heteromeric=True, pathway_sum=True)

NameError: name 'ct' is not defined

In [ ]:
ct.tl.spatial_communication(adata_chuv_dis500,
    database_name='cellchat', df_ligrec=df_cellchat_filtered_chuv, dis_thr=500, heteromeric=True, pathway_sum=True)

In [13]:
#Reading the results from the commot analysis -> took about 20 minutes to run for L1_1 and chuvio_L1_1 

In [56]:
path_adata_chuv_dis500 = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/Analysis/Commot/adata_chuv_dis500.h5ad"
path_adata_dis500 = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/Analysis/Commot/adata_dis500.h5ad"

adata_chuv_dis500_commot = sc.read_h5ad(path_adata_chuv_dis500)
adata_dis500_commot = sc.read_h5ad(path_adata_dis500)


#### OUT checking run time for commot

In [9]:
import os
import time

# Path to the file/directory
path = path_adata_chuv_dis500

# Both the variables would contain time
# elapsed since EPOCH in float
ti_c = os.path.getctime(path)
ti_m = os.path.getmtime(path)

# Converting the time in seconds to a timestamp
c_ti = time.ctime(ti_c)
m_ti = time.ctime(ti_m)

print(f"The file located at the path {path} \
was created at {c_ti} and was "
      f"last modified at {m_ti}")

The file located at the path /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/Analysis/Commot/adata_chuv_dis500.h5ad was created at Thu Apr 25 11:43:14 2024 and was last modified at Thu Apr 25 11:43:14 2024


# Analysing commot results 

In [66]:
path_adata_chuv_dis500 = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/Analysis/Commot/adata_chuv_dis500.h5ad"
path_adata_dis500 = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/Analysis/Commot/adata_dis500.h5ad"

adata_chuv_dis500_commot = sc.read_h5ad(path_adata_chuv_dis500)
adata_dis500_commot = sc.read_h5ad(path_adata_dis500)

In [84]:
ct.tl.communication_direction(adata_chuv_dis500_commot, database_name='cellchat', pathway_name='CXCL', k=5)

In [74]:
adata_chuv_dis500_commot

AnnData object with n_obs × n_vars = 71187 × 313
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'libsize_drop', 'sizeFactor', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'singler_annotation', 'ident'
    uns: 'X_name', 'commot-cellchat-info'
    obsm: 'PCA', 'TSNE', 'UMAP', 'commot-cellchat-sum-receiver', 'commot-cellchat-sum-sender', 'spatial'
    layers: 'logcounts'
    obsp: 'commot-cellchat-AREG-EGFR', 'commot-cellchat-CXCL', 'commot-cellchat-CXCL13-CXCR5', 'commot-cellchat-EGF', 'commot-cellchat-FASLG', 'commot-cellchat-FASLG-FAS', 'commot-cellchat-total-total'

In [86]:
ct.pl.plot_cell_communication(adata_dis500, database_name='cellchat', pathway_name='adata_chuv_dis500_commot', plot_method='grid', background_legend=False,
    scale=0.00009, ndsize=12, grid_density=0.3, normalize_v = True, normalize_v_quantile=0.995)

KeyError: 'commot_sender_vf-cellchat-adata_chuv_dis500_commot'

In [ ]:
chuvio_L1_1.obs['leiden'] = adata.obs['leiden']

In [ ]:
#Get the main gene interactions 
#